In [ ]:
from transaction_analysis.profit_scanner import ProfitScanner
from utils.transaction.swap_transaction_analyzer import *

from utils.transaction.erc20_transaction import ERC20Transaction
from utils.transaction.normal_transaction import NormalTransaction
from utils.transaction.internal_transaction import InternalTransaction

In [ ]:
#enable autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
address = "0x1a869357c99bdf201D0378971FfE0b78DA5697E0"

profit_scanner = ProfitScanner(address)

In [ ]:
txs_by_hash = profit_scanner.create_txs_by_hash()
function_name_by_hash = profit_scanner.create_function_name_by_hash()

In [ ]:
tt = sorted(list(set(function_name_by_hash.values())))

In [ ]:
tt

In [ ]:
from collections import defaultdict

d = set()

for tx_hash in txs_by_hash:
    if tx_hash not in function_name_by_hash:
        continue
    if function_name_by_hash[tx_hash] == SwapFunctionName.swap_exact_tokens_for_eth_supporting_fee_on_transfer_tokens.value:
        cnt_by_tx_name = defaultdict(int)
        for tx in txs_by_hash[tx_hash]:
            cnt_by_tx_name[tx.transaction_name] += 1
        if cnt_by_tx_name["erc20_transaction"] == 2:
            print(tx_hash)
        cnt_by_tx_tuple = tuple(sorted(cnt_by_tx_name.items()))
        d.add(cnt_by_tx_tuple)

d

In [ ]:
tx_hash = "0x6315c2f7ec57c3a63ee7020ef8785c596963eaf7ff37c3b95d98b465aede3d25"

def get_bundle_by_tx_hash(tx_hash):
    txs = txs_by_hash[tx_hash]

    normal_transaction, erc20_transactions, internal_transaction = None, [], None

    for tx in txs:
        if type(tx) == NormalTransaction:
            normal_transaction = tx
        elif type(tx) == ERC20Transaction:
            erc20_transactions.append(tx)
        elif type(tx) == InternalTransaction:
            internal_transaction = tx
        else:
            raise Exception("Unknown transaction type")

    return normal_transaction, erc20_transactions, internal_transaction

normal_transaction, erc20_transactions, internal_transaction = get_bundle_by_tx_hash(tx_hash)

In [ ]:
txs_by_hash["0x7c1bcddc647379752a2e93b3d89e730d410f541f1626d362212dd641b643703f"]